In [2]:
#%pip install torch --index-url https://download.pytorch.org/whl/cpu
%pip install datasets python-dotenv

from utils import *


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [10]:

import random
from datasets import load_dataset, Dataset
import json
from openai.types.chat.chat_completion import ChatCompletion
import os

process_details_schema = {
    "type": "object",
    "properties": {
        "description": {
            "type": "string",
            "description": "A detailed description of the process, when it is done, how it is done, and why it is done, and what its outcome should be."
        },
        "location1": {
            "type": "string",
            "examples": ["indoor", "outdoor", "indoor/outdoor"],
            "description": "The location where the process is done primarily."
        },
        "location2": {
            "type": "string",
            "examples": ["office", "cubical", "desk", "vehicle", "factory", "site"],
            "description": "The location where the process is done primarily."
        },
        "location_name": {
            "type": "string",
            "examples": ["clinic", "school", "plane"],
            "description": "The name of the location where the process is done primarily."
        },
        "challenge": {
            "type": "string",
            "examples": ["physical", "mental", "verbal", "emotional", "spiritual"],
            "description": "The main challenge that the process presents."
        },
        "duration": {
            "type": "string",
            "examples": ["1m", "30m", "1h", "3d"],
            "description": "The duration of the process."
        },
        "frequency": {
            "type": "string",
            "examples": ["continuous", "hourly", "daily", "weekly", "monthly", "yearly"],
            "description": "The frequency of the process."
        },
        "risks": {
            "type": "array",
            "description": "A list of risks associated with the process.",
            "items": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "examples": ["fall", "injury", "accident", "illness"],
                        "description": "The name of the risk."
                    },
                    "description": {
                        "type": "string",
                        "description": "A detailed description of the risk and how it can be mitigated."
                    }
                }
            }
        },
        "equipment": {
            "type": "array",
            "description": "A list of equipment that is used in the process.",
            "items": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "examples": ["stethoscope", "laptop", "screwdriver"],
                        "description": "The name of the equipment."
                    },
                    "description": {
                        "type": "string",
                        "description": "A detailed description of the equipment and how it is used."
                    }
                }
            }
        },
        "localities": {
            "type": "array",
            "description": "A list of all localities involved in the process.",
            "items": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "examples": ["treatment room", "classroom", "passenger cabin"],
                        "description": "The name of the locality."
                    },
                    "description": {
                        "type": "string",
                        "description": "A detailed description of the locality and how it is involved."
                    },
                }
            }
        },
        "sourroundings": {
            "type": "array",
            "description": "A list of all sourroundings supporting in the process.",
            "items": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "examples": ["waiting room", "playground", "kitchen"],
                        "description": "The name of the sourrounding."
                    },
                    "description": {
                        "type": "string",
                        "description": "A detailed description of the sourrounding and how it supports the process."
                    },
                }
            }
        },
        "people": {
            "type": "array",
            "description": "A list of all people involved in the process.",
            "items": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "examples": ["patient", "student", "passenger"],
                        "description": "The name of the person."
                    },
                    "description": {
                        "type": "string",
                        "description": "A detailed description of the person and how they are involved."
                    },
                }
            }
        },
    },
    "required": ["name", "description", "location1", "location2", "location_name", "challenge", "duration", "frequency", "risks", "equipment", "localities", "sourroundings", "people"],
    "additionalProperties": False
}

def generate_process_details(entry):
    results = []

    available_models = [
        "mistralai/mistral-nemo",
        #"mistralai/mixtral-8x7b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mistral-small",
        #"mistralai/mistral-medium",
        "mistralai/mistral-large"
    ]

    model = random.choice(available_models)

    openrouter_response: ChatCompletion = openrouter.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": f"You are an expert {entry['profession']}. You are tasked with describing your job and all that it entails."
        }, {
            "role": "user",
            "content": f"Generate a detailed description for the profession {entry['profession']} in the process {entry['process']}. "
        }],
        tools=[{
            "type": "function",
            "function": {
                "name": "generate_process_details",
                "parameters": process_details_schema,
            },
        }],
        tool_choice="required",
        max_tokens=8191,
        temperature=1.2,
        extra_body={
            "provider": {
                "require_parameters": True,
            },
        }
    )

    process_detail = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)

    results.append({
        "profession": entry["profession"],
        "process": entry["process"],
        "description": process_detail["description"],
        "location1": process_detail["location1"],
        "location2": process_detail["location2"],
        "location_name": process_detail["location_name"],
        "challenge": process_detail["challenge"],
        "duration": process_detail["duration"],
        "frequency": process_detail["frequency"],
        "risks": process_detail["risks"],
        "equipment": process_detail["equipment"],
        "localities": process_detail["localities"],
        "sourroundings": process_detail["sourroundings"],
        "people": process_detail["people"],
    })
    
    return results


processes = load_dataset("lucaelin/generic_processes_v1", split="train")
domain_actions = processes.shuffle().map(
    sane_batch_map(generate_process_details, {"profession": [], "process": [], "description": [], "location1": [], "location2": [], "location_name": [], "challenge": [], "duration": [], "frequency": [], "importance": [], "equipment": [], "localities": [], "sourroundings": [], "people": []}),
    batched=True, 
    batch_size=4, 
    num_proc=4, 
    remove_columns=processes.column_names
)
#domain_actions.sort(["profession", "process"]).to_list()
domain_actions.sort(["profession", "process"]).push_to_hub("lucaelin/generic_process_details_v1")

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

{'profession': 'Receptionist', 'process': 'Administrative Support'}
Traceback (most recent call last):
  File "/tmp/ipykernel_2349765/420438122.py", line 20, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_2349765/1408341564.py", line 196, in generate_process_details
    "description": process_detail["description"],
                   ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^
KeyError: 'description'



Map (num_proc=4):   2%|▏         | 24/1000 [01:32<51:20,  3.16s/ examples]  

{'profession': 'Sports Coach', 'process': 'Equipment Management'}
Traceback (most recent call last):
  File "/tmp/ipykernel_2349765/420438122.py", line 20, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_2349765/1408341564.py", line 196, in generate_process_details
    "description": process_detail["description"],
                   ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^
KeyError: 'description'

{'profession': 'Financial Advisor', 'process': 'Market Research'}
Traceback (most recent call last):
  File "/tmp/ipykernel_2349765/420438122.py", line 20, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_2349765/1408341564.py", line 206, in generate_process_details
    "sourroundings": process_detail["sourroundings"],
                     ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
KeyError: 'sourroundings'



Map (num_proc=4):   3%|▎         | 28/1000 [02:00<1:10:53,  4.38s/ examples]

...

{'profession': 'Loan Officer', 'process': 'Loan Repayment Monitoring'}
Traceback (most recent call last):
  File "/tmp/ipykernel_2349765/420438122.py", line 20, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_2349765/1408341564.py", line 196, in generate_process_details
    "description": process_detail["description"],
                   ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^
KeyError: 'description'



Map (num_proc=4):  96%|█████████▋| 964/1000 [48:29<03:40,  6.14s/ examples]

{'profession': 'Registered Nurse', 'process': 'Wound Care'}
Traceback (most recent call last):
  File "/tmp/ipykernel_2349765/420438122.py", line 20, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_2349765/1408341564.py", line 196, in generate_process_details
    "description": process_detail["description"],
                   ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^
KeyError: 'description'



Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.72s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lucaelin/generic_process_details_v1/commit/bd6c710320c3ede56abaa33a58d9842864234392', commit_message='Upload dataset', commit_description='', oid='bd6c710320c3ede56abaa33a58d9842864234392', pr_url=None, pr_revision=None, pr_num=None)